## Supervised training
Refer to 
https://github.com/jalkestrup/llm2vec-dtu/blob/main/experiments/run_supervised.py

In [4]:
import os
from datasets import load_dataset
import transformers
from tqdm import tqdm
from accelerate import Accelerator, DistributedDataParallelKwargs

In [5]:
from huggingface_hub import HfApi
from dotenv import load_dotenv
load_dotenv()

api = HfApi(token=os.getenv("HF_TOKEN"))

# Alternatively, login with huggingface_hub GUI
#notebook_login()

# Handle lighting AI studio path
if '/teamspace' in os.getcwd():
    os.chdir('/teamspace/studios/this_studio/llm2vec-da')
    # Hmm lighting AI studio changed to the below ..?
    #os.chdir('/home/zeus/content/llm2vec-da')
    print(os.getcwd())

/teamspace/studios/this_studio/llm2vec-da


In [6]:
from transformers import HfArgumentParser, TrainingArguments
from llm2vec_da.arguments import EmbeddingModelArguments, DataTrainingArguments, CustomArguments

supervised_parser = HfArgumentParser(
        (EmbeddingModelArguments, DataTrainingArguments, TrainingArguments, CustomArguments)
    )

model_args, data_args, training_args, custom_args = supervised_parser.parse_json_file(
        "configs/supervised/MetaLlama3-sheared.json"
    )

if training_args.ddp_find_unused_parameters:
    kwargs = [
        DistributedDataParallelKwargs(
            dim=0,
            broadcast_buffers=True,
            bucket_cap_mb=25,
            find_unused_parameters=True,
            check_reduction=False,
            gradient_as_bucket_view=False,
        )
    ]
else:
    kwargs = []

accelerator = Accelerator(kwargs_handlers=kwargs)
transformers.set_seed(training_args.seed)

#ABSOLUTELY CRITICAL OR WILL CAUSE OBSCURE NO GRAD ERROR THAT TOOK FREAKING 4 HOURS TO IDENTIFY
if training_args.gradient_checkpointing:
    training_args.gradient_checkpointing_kwargs = {"use_reentrant": False}

## Load data

In [7]:
from dataclasses import dataclass
from typing import List, Union
import random
import logging
import torch

logger = logging.getLogger(__name__)

class Dataset(torch.utils.data.Dataset):
    """
    Abstract class for datasets
    """
    
    def load_data(self, file_path: str = None):
        raise NotImplementedError()

    def __getitem__(self, index):
        raise NotImplementedError()

    def __len__(self):
        raise NotImplementedError()

@dataclass
class DataSample:
    id_: int
    query: str
    positive: str
    negative: str = None
    task_name: str = None


class TrainSample:
    """
    Structure for one input example with texts, the label and a unique id
    """

    def __init__(
        self, guid: str = "", texts: List[str] = None, label: Union[int, float] = 0
    ):
        """
        Creates one TrainSample with the given texts, guid and label


        :param guid
            id for the example
        :param texts
            the texts for the example.
        :param label
            the label for the example
        """
        self.guid = guid
        self.texts = texts
        self.label = label

    def __str__(self):
        return "<TrainSample> label: {}, texts: {}".format(
            str(self.label), "; ".join(self.texts)
        )


In [8]:
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


class NordicE5Data(Dataset):
    """
    A dataset class for loading and processing data from a Hugging Face dataset to a datasample following E5 datastructure.
    
    This class handles loading instruction-based samples with queries, positive examples,
    and optional negative examples. It processes the data into batches suitable for training.

    Args:
        hf_dataset: The dataset to load from (local or remote)
        [Optional] instruction_column (str): Column name for instructions. Defaults to 'instruction'. Prepends the instruction to the query.
        query_column (str): Column name for queries. Defaults to 'query'
        pos_column (str): Column name for positive examples. Defaults to 'positive'
        [Optional] neg_column (str): Column name for negative examples. Defaults to 'negative'
        [Optional] task_column (str): Column name for task labels. Task is used to group the data by task during batching.
        split (str): Dataset split to use. Defaults to "train"
        effective_batch_size (int): Size of batches to create, accounting for parallel processes.
        separator (str): Separator string between text segments. Defaults to "!@#$%^&*()"
    """

    def __init__(
        self,
        hf_dataset,
        instruction_column = 'instruction',
        query_column = 'query',
        pos_column = 'positive',
        neg_column = 'negative',
        task_column = 'task',
        split: str = "train",
        effective_batch_size: int = 32,
        separator: str = "!@#$%^&*()", #Note default of LLM2Vec is !@#$%^&*() , changing this would also have to be changed in the llm2vec lib when encoding/decoding
    ):
        self.instruction_column = instruction_column
        self.query_column = query_column
        self.pos_column = pos_column
        self.neg_column = neg_column
        self.task_column = task_column
        self.split = split
        self.effective_batch_size = effective_batch_size
        self.separator = separator
        self.data = []
        self.load_data(hf_dataset)

    def __len__(self):
        return len(self.data)

    def load_data(self, hf_dataset):
        # 1) Convert the hf dataset to a list of DataSamples
        all_samples = []
        for idx, row in tqdm(enumerate(hf_dataset), total=len(hf_dataset), desc='Loading dataset'):
            
            # If no query and positive example, skip the example
            if self.query_column not in row or self.pos_column not in row:
                logger.warning(f"No query or positive example found for example {idx}, skipping")
                continue

            # If instruction column is provided, prepend the instruction to the query
            if self.instruction_column:
                instruction = row[self.instruction_column]
                query =  f"{instruction}; {self.separator}{row[self.query_column]}"
            else:
                query =  f"{row[self.query_column]}"
        
            pos   =  f"{self.separator}{row[self.pos_column]}"

            # If negative column is provided include negative example
            neg_raw = row[self.neg_column]
            if neg_raw is None or neg_raw.strip().lower() in {"", "none", "null"}:
                neg = None
            else:
                neg   =  f"{self.separator}{row[self.neg_column]}"

            # If task column is provided include task name as to group batches per task
            if row[self.task_column]:
                task  =  row[self.task_column]
            else:
                task = None

            all_samples.append(
                DataSample(
                    id_=idx,
                    query=query,
                    positive=pos,
                    negative=neg,
                    task_name=task
                )
            )

        # First, group samples by task
        task_samples = {}
        for idx, sample in tqdm(enumerate(all_samples), total=len(all_samples), desc='Grouping data by task'):
            task = sample.task_name
            if task not in task_samples:
                task_samples[task] = []
            task_samples[task].append(sample)

        logger.info(f"Batching data for effective batch size = {self.effective_batch_size} ...")
        batched_data = []

        # Create full batches for each task
        for task, samples in tqdm(task_samples.items(), total=len(task_samples), desc='Batching data'):
            task_batches = []
            for i in range(0, len(samples), self.effective_batch_size):
                batch = samples[i : i + self.effective_batch_size]
                if len(batch) == self.effective_batch_size:
                    task_batches.append(batch)
                else:
                    logger.info(f"Skipping partial batch of {len(batch)} samples for task {task}")
            
            if task_batches:  # If we got any full batches for this task
                batched_data.extend(task_batches)

        # Shuffle the batches to mix tasks during training
        random.shuffle(batched_data)

        # Flatten while maintaining batch boundaries
        self.data = [sample for batch in batched_data for sample in batch]
        logger.info(f"Loaded and batched {len(self.data)} samples from {len(task_samples)} tasks")

    def __getitem__(self, index):
        sample = self.data[index]
        texts = [sample.query, sample.positive]
        if sample.negative is not None:          
            texts.append(sample.negative)
        return TrainSample(texts=texts, label=1.0)
        
def custom_dataset(hf_dataset,
                      effective_batch_size):
    
    dataset_map = {
        "nordic-embedding-training-data": NordicE5Data
    }

    if hf_dataset.info.dataset_name in dataset_map:
        return dataset_map[hf_dataset.info.dataset_name](hf_dataset,
                                                        effective_batch_size=effective_batch_size)
    else:
        raise ValueError(f"Dataset {hf_dataset.info.dataset_name} not found in dataset_map")


In [9]:
dataset = load_dataset(data_args.dataset_name,
split="train[:10%]")
#,columns=['query', 'positive', 'negative', 'instruction', 'task'])

# Optionally, save to local file
#dataset.save_to_disk("nordic-embedding-training-data")

# Optionally, load from local file
#from datasets import load_from_disk
#ds_transformed = load_from_disk("/teamspace/studios/this_studio/synthetic-supervised-dataset-2")

print(dataset[0])

# Split the dataset into 95% train and 5% test
split_dataset = dataset.train_test_split(test_size=0.05, seed=42)

# Define the splits
train_dataset = split_dataset['train']
valid_dataset = split_dataset['test']

{'query': 'Hvad var de langsigtede konsekvenser for dansk økonomi af den danske forfatningslov af 1849?', 'positive': 'Den danske forfatningslov af 1849 markerede et paradigmeskift i dansk politisk og økonomisk struktur.  Loven etableret et konstitutionelt monarki, begrænsede kongelig magt og indførte en parlamentarisk form for styre. Dette havde vidtrækkende konsekvenser for den danske økonomi.  Den nye forfatning lagde grunden for en mere liberalistisk økonomisk orden, med fokus på fri handel, privat ejendomsret og entreprenørskab.  Den øgede politiske stabilitet og forudsigelighed tiltrak udenlandsk kapital og investeringer, der bidrog til økonomisk vækst.  Samtidig reducerede loven den statslige indblanding i økonomien, hvilket gav plads til privat initiativ og markedskræfter.  Introduktionen af en national valuta og en centralbank styrkede den økonomiske integration med andre europæiske lande.  Mens forfatningsloven af 1849 ikke direkte førte til økonomisk mirakel, lagde den grund

In [10]:
train_dataset_e5 = custom_dataset(train_dataset, 
                                 effective_batch_size=training_args.per_device_train_batch_size* accelerator.num_processes)

valid_dataset_e5 = custom_dataset(valid_dataset,
                                effective_batch_size=training_args.per_device_train_batch_size* accelerator.num_processes)

Grouping data by task: 100%|██████████| 91983/91983 [00:00<00:00, 2248842.45it/s]
INFO:__main__:Batching data for effective batch size = 32 ...
Batching data:   0%|          | 0/2 [00:00<?, ?it/s]INFO:__main__:Skipping partial batch of 21 samples for task retrieval
INFO:__main__:Skipping partial batch of 26 samples for task classification
Batching data: 100%|██████████| 2/2 [00:00<00:00, 382.88it/s]
INFO:__main__:Loaded and batched 91936 samples from 2 tasks
Grouping data by task: 100%|██████████| 4842/4842 [00:00<00:00, 2702078.23it/s]
INFO:__main__:Batching data for effective batch size = 32 ...
Batching data:   0%|          | 0/2 [00:00<?, ?it/s]INFO:__main__:Skipping partial batch of 26 samples for task retrieval
INFO:__main__:Skipping partial batch of 16 samples for task classification
Batching data: 100%|██████████| 2/2 [00:00<00:00, 2810.25it/s]
INFO:__main__:Loaded and batched 4800 samples from 2 tasks


In [11]:
train_dataset_e5[0].texts

['Locate job postings requiring a specific skill set and industry experience.; !@#$%^&*()Jeg søger en stilling som postdoc forsker indenfor kvantitativ genetik med fokus på genomisk selektion hos hvede, hvor jeg kan kombinere mineOoh ekspertise i statistisk modellering med brugen af moderne højtydende computer clusters.',
 '!@#$%^&*()Vi søger en motiveret og dygtig postdoc forsker til at slutte sig til vores hold, der arbejder med at forstå de genetiske mekanismer bag udvikling af acidotolerante hvedesorter. Projektet involverer anvendelsen af genomisk selektionsanalyse, kvantitativ genetik og avanceret statistisk modellering. Du vil have adgang til state-of-the-art computerclusters og arbejde med store datasæt af sekvensdata. Kendskab til programmeringssprog som R eller Python er et krav. En afhandling indenfor et relevant felt, f.eks. genetik, bioinformatik eller matematik, forventes.',
 '!@#$%^&*()Vores team af forskere er specialiseret i anvendelse af maskinlæring til at analysere 

In [12]:
def prepare_for_tokenization(model, text, pooling_mode="mean"):
    if model.config._name_or_path == "meta-llama/Meta-Llama-3-8B-Instruct":
        text = (
            "<|start_header_id|>user<|end_header_id|>\n\n" + text.strip() + "<|eot_id|>"
        )
        return text
    if model.config._name_or_path in [
        "mistralai/Mistral-7B-Instruct-v0.2",
        "meta-llama/Llama-2-7b-chat-hf",
    ]:
        text = "[INST] " + text.strip() + " [/INST]"
    if model.config._name_or_path in [
        "google/gemma-2-9b-it",
    ]:
        text = "<bos><start_of_turn>user\n" + text.strip() + "<end_of_turn>"
    if model.config._name_or_path in [
        "Qwen/Qwen2-1.5B-Instruct",
        "Qwen/Qwen2-7B-Instruct",
    ]:
        text = "<|im_start|>user\n" + text.strip() + "<|im_end|>"
    if pooling_mode == "eos_token":
        if model.config._name_or_path == "meta-llama/Meta-Llama-3-8B":
            text = text.strip() + "<|end_of_text|>"
        elif isinstance(model.config, LlamaConfig) or isinstance(
            model.config, MistralConfig
        ):
            text = text.strip() + " </s>"
        elif isinstance(model.config, GemmaConfig):
            text = text.strip() + "<eos>"
        elif isinstance(model.config, Qwen2Config):
            text = text.strip() + "<|endoftext|>"
    return text

class MixedNegCollator:
    #def __init__(self, model: LLM2Vec):
    def __init__(self, model):
        self.model = model

    def _prep(self, txt):
        return prepare_for_tokenization(self.model, txt,
                                        pooling_mode=self.model.pooling_mode)

    def __call__(self, batch):
        q_texts, p_texts, n_texts, labels = [], [], [], []

        for ex in batch:
            q_texts.append(self._prep(ex.texts[0]))
            p_texts.append(self._prep(ex.texts[1]))

            if len(ex.texts) > 2 and ex.texts[2]:
                n_texts.append(self._prep(ex.texts[2]))

            labels.append(ex.label)

        sent_feat_q = self.model.tokenize(q_texts)          # size B
        sent_feat_p = self.model.tokenize(p_texts)          # size B
        sent_feat_n = (
            self.model.tokenize(n_texts) if n_texts else None
        )      
                                                     # size ≤ B or None

        return [sent_feat_q, sent_feat_p, sent_feat_n], torch.tensor(labels)

## Tokenizer test

In [13]:
from transformers import (
    LlamaConfig,
    MistralConfig,
    GemmaConfig,
    Qwen2Config,
)

In [14]:
# import torch
# from torch import nn
# from transformers import AutoTokenizer, AutoModel
# from transformers import LlamaConfig


# class TinyLLM2Vec(nn.Module):
#     """
#     Drop‑in replacement for LLM2Vec that is tiny but respects the API:
#       - .tokenize(list[str]) -> dict[str, Tensor] batch encoding
#       - .encode(features)    -> Tensor (batch, D)
#       - .pooling_mode attr   -> str
#     """
#     def __init__(self, model_name="prajjwal1/bert-tiny", pooling_mode="cls"):
#         super().__init__()
#         self.tokenizer = AutoTokenizer.from_pretrained(model_name)
#         self.model     = AutoModel.from_pretrained(model_name)
#         self.config = self.model.config          # forward attr used by prep‑fn
#         #self.config = LlamaConfig()
#         self.config._name_or_path = "meta-llama/Meta-Llama-3-8B" # To fake the config
        
#         self.pooling_mode = pooling_mode   # value read by prepare_for_tokenization

#     @torch.no_grad()
#     def tokenize(self, texts):
#         return self.tokenizer(
#             texts, padding=True, truncation=True, return_tensors="pt", max_length=512
#         )

#     @torch.no_grad()
#     def encode(self, features):
#         out = self.model(**features).last_hidden_state   # (B, L, H)
#         if self.pooling_mode == "cls":
#             return out[:, 0]                             # (B, H)
#         elif self.pooling_mode == "mean":
#             mask = features["attention_mask"].unsqueeze(-1)
#             return (out * mask).sum(1) / mask.sum(1)     # (B, H)
#         else:
#             raise ValueError("Unknown pooling mode")

# model_tiny = TinyLLM2Vec(pooling_mode="mean")      # instead of Llama‑8B

### Test of prepare_for_tokenization

In [15]:
# # Set _name_or_path to define tokenizer model behavior
# model_tiny.config._name_or_path =  "meta-llama/Meta-Llama-3-8B"

# # Inspect the input query and the output query before and after
# print(f'Input query: {train_dataset_e5[0].texts[0]}')
# print(f'Output query: {prepare_for_tokenization(model_tiny, train_dataset_e5[0].texts[0], pooling_mode="eos_token")}')

In [16]:
# from llm2vec_da.loss import HardNegativeNLLLoss
# from torch.utils.data import DataLoader

# collator = MixedNegCollator(model_tiny)           # the new collator

# loader   = DataLoader(
#                dataset=train_dataset_e5,
#                batch_size=32,                 
#                shuffle=False, # DO NOT SHUFFLE, batching is done in the dataset class
#                collate_fn=collator
#            )

# loss_fn  = HardNegativeNLLLoss(scale=20.0)   # unchanged

In [17]:
# batch = next(iter(loader))
# (q_feat, p_feat, n_feat), _ = batch

# q_reps = model_tiny.encode(q_feat)                # (B, D)
# p_reps = model_tiny.encode(p_feat)                # (B, D)
# n_reps = model_tiny.encode(n_feat) if n_feat else None

# loss = loss_fn(q_reps, p_reps, n_reps)
# print("forward OK, loss =", loss.item())

## Load model

In [18]:
from llm2vec_da import LLM2Vec

torch_dtype = (
    model_args.torch_dtype
    if model_args.torch_dtype in ["auto", None]
    else getattr(torch, model_args.torch_dtype)
)

#training_args.gradient_checkpointing = False   # turn it off
model = LLM2Vec.from_pretrained(
    base_model_name_or_path=model_args.model_name_or_path,
    enable_bidirectional=model_args.bidirectional,
    peft_model_name_or_path="McGill-NLP/LLM2Vec-Sheared-LLaMA-mntp",
    merge_peft=True,
    pooling_mode=model_args.pooling_mode,
    max_length=data_args.max_seq_length,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=model_args.low_cpu_mem_usage,
    attn_implementation="sdpa", #OBS SET BACK TO FLASH ATTENTION WHEN RUNNING ON A100 GPU!!
)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message.


## Set up PEFT

In [19]:
from llm2vec_da.model import initialize_peft


peft_model = initialize_peft(
    model.model,
    lora_r=custom_args.lora_r,
    lora_alpha=2 * custom_args.lora_r,
    lora_dropout=custom_args.lora_dropout,
)

# model organization is LLM2VecModel.model -> HF Model, we have to apply PEFT to the inner model
model.model = peft_model.model

INFO:peft.tuners.tuners_utils:Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!


Model's Lora trainable parameters:
trainable params: 14,991,360 || all params: 1,294,878,720 || trainable%: 1.1577


In [20]:
print("after-peft trainable:",
      [n for n,p in model.named_parameters() if p.requires_grad][:10])

after-peft trainable: ['model.layers.0.self_attn.q_proj.lora_A.default.weight', 'model.layers.0.self_attn.q_proj.lora_B.default.weight', 'model.layers.0.self_attn.k_proj.lora_A.default.weight', 'model.layers.0.self_attn.k_proj.lora_B.default.weight', 'model.layers.0.self_attn.v_proj.lora_A.default.weight', 'model.layers.0.self_attn.v_proj.lora_B.default.weight', 'model.layers.0.self_attn.o_proj.lora_A.default.weight', 'model.layers.0.self_attn.o_proj.lora_B.default.weight', 'model.layers.0.mlp.gate_proj.lora_A.default.weight', 'model.layers.0.mlp.gate_proj.lora_B.default.weight']


In [21]:
tok = model.tokenize(["just a test"])              # CPU tensors
dev = next(model.parameters()).device              # cuda:0 or cpu
tok = {k: v.to(dev) for k, v in tok.items()}       # move batch

with torch.set_grad_enabled(True):
    reps = model(tok)                              # runs __call__ → forward
print("requires_grad:", reps.requires_grad, reps.grad_fn)

requires_grad: True <StackBackward0 object at 0x7f481be80c70>


## Training

In [22]:
#from llm2vec_da.training import SupervisedDefaultCollator
tokenizer = model.tokenizer
data_collator = MixedNegCollator(model)           # the new collator

# Load train examples into memory
train_examples = [
    train_dataset_e5[i]
    for i in tqdm(
        range(len(train_dataset_e5)),
        desc="Loading train examples...",
        disable=not accelerator.is_main_process,
    )
]

valid_examples = [
    valid_dataset_e5[i]
    for i in tqdm(
        range(len(valid_dataset_e5)),
        desc="Loading valid examples...",
        disable=not accelerator.is_main_process,
    )
]

Loading valid examples...: 100%|██████████| 4800/4800 [00:00<00:00, 442106.79it/s]


In [28]:
from transformers import Trainer
from typing import Any, Optional, Tuple, Dict, Union
from torch.utils.data import DataLoader, SequentialSampler
from transformers.trainer_utils import seed_worker
from torch import nn


class SupervisedTrainer(Trainer):
    def __init__(
        self,
        *args,
        loss_function=None,
        **kwargs,
    ) -> None:
        super().__init__(*args, **kwargs)
        self.loss_function = loss_function

    def compute_loss(
        self,
        model: nn.Module,
        inputs: Dict[str, Union[torch.Tensor, Any]],
        return_outputs: bool = False,
        **kwargs,                      # <-- swallow future extras
    ) -> Union[Tuple[torch.Tensor, torch.Tensor], torch.Tensor]:

        features, labels = inputs
        q_reps = self.model.forward(features[0])
        d_reps = self.model.forward(features[1])

        d_reps_neg = None
        if len(features) > 2 and features[2] is not None:
            d_reps_neg = self.model.forward(features[2])

        loss = self.loss_function(q_reps, d_reps, d_reps_neg)

        if return_outputs:
            output = torch.cat(
                [model(row)["sentence_embedding"][:, None] for row in features], dim=1
            )
            return loss, output

        return loss

    def get_train_dataloader(self) -> DataLoader:
        # Copying most of the code from the parent class, changing the sampler to SequentialSampler
        if self.train_dataset is None:
            raise ValueError("Trainer: training requires a train_dataset.")

        train_dataset = self.train_dataset
        data_collator = self.data_collator

        data_collator = self._get_collator_with_removed_columns(
            data_collator, description="training"
        )

        dataloader_params = {
            "batch_size": self._train_batch_size,
            "collate_fn": data_collator,
            "num_workers": self.args.dataloader_num_workers,
            "pin_memory": self.args.dataloader_pin_memory,
            "persistent_workers": self.args.dataloader_persistent_workers,
        }

        if not isinstance(train_dataset, torch.utils.data.IterableDataset):
            # Changing from random sampler to sequential sampler
            dataloader_params["sampler"] = SequentialSampler(train_dataset)
            dataloader_params["drop_last"] = self.args.dataloader_drop_last
            dataloader_params["worker_init_fn"] = seed_worker

        return self.accelerator.prepare(DataLoader(train_dataset, **dataloader_params))

    def _save(self, output_dir: Optional[str] = None, state_dict=None):
        # If we are executing this function, we are the process zero, so we don't check for that.
        output_dir = output_dir if output_dir is not None else self.args.output_dir
        os.makedirs(output_dir, exist_ok=True)
        logger.info(f"Saving model checkpoint to {output_dir}")

        self.model.save(output_dir)

        # Good practice: save your training arguments together with the trained model
        torch.save(self.args, os.path.join(output_dir, "training_args.bin"))

In [30]:
from llm2vec_da.training import StopTrainingCallback
from llm2vec_da.loss.utils import load_loss

train_loss = load_loss(custom_args.loss_class, scale=custom_args.loss_scale)

samples_skipped = 40 * training_args.per_device_train_batch_size   # 40 steps × 32-sample batches
train_subset    = train_examples[samples_skipped:]  

trainer = SupervisedTrainer(
    model=model,
    args=training_args,
    train_dataset=train_subset,
    eval_dataset=valid_examples,
    data_collator=data_collator,
    tokenizer=model.tokenizer,
    loss_function=train_loss,
)

if custom_args.stop_after_n_steps is not None:
    trainer.add_callback(StopTrainingCallback(custom_args.stop_after_n_steps))

/tmp/ipykernel_830075/3893196380.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `SupervisedTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


In [25]:
# os.environ["WANDB_DISABLED"] = "false"

In [31]:
trainer.train()

***** Running training *****
  Num examples = 90,656
  Num Epochs = 3
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 8,499
  Number of trainable parameters = 14,991,360
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Could not log the number of model parameters in Weights & Biases due to an AttributeError.


Step,Training Loss
1,0.972700
5,5.754900
10,0.994500
15,1.018800
20,1.093000
25,0.973400
30,5.320300
35,0.782800
40,0.568400
45,0.543000


INFO:__main__:Saving model checkpoint to output/mntp-supervised/Meta-Llama-3-sheared/checkpoint-50
Configuration saved in output/mntp-supervised/Meta-Llama-3-sheared/checkpoint-50/config.json
Model weights saved in output/mntp-supervised/Meta-Llama-3-sheared/checkpoint-50/model.safetensors
tokenizer config file saved in output/mntp-supervised/Meta-Llama-3-sheared/checkpoint-50/tokenizer_config.json
Special tokens file saved in output/mntp-supervised/Meta-Llama-3-sheared/checkpoint-50/special_tokens_map.json
INFO:__main__:Saving model checkpoint to output/mntp-supervised/Meta-Llama-3-sheared/checkpoint-100
Configuration saved in output/mntp-supervised/Meta-Llama-3-sheared/checkpoint-100/config.json
Model weights saved in output/mntp-supervised/Meta-Llama-3-sheared/checkpoint-100/model.safetensors
tokenizer config file saved in output/mntp-supervised/Meta-Llama-3-sheared/checkpoint-100/tokenizer_config.json
Special tokens file saved in output/mntp-supervised/Meta-Llama-3-sheared/checkpoi

KeyboardInterrupt: 